# Python Connect Online DB 

In [2]:
import mysql.connector

# Creating connection object
mydb_conn = mysql.connector.connect(
    host = "mysql-249318cb-zeyadmedhat-4161.f.aivencloud.com",
    user = "avnadmin",
    password = "AVNS_tQUDqMdUkSIiLIYFWMp",
    port=18607,
    database='library'
)

# Printing the connection object
print(mydb_conn)


In [3]:
cursor = mydb_conn.cursor()
cursor

In [4]:
# /******************* In the Library *********************/

# /*******************************************************/
# /* find the number of availalbe copies of the book (Dracula)      */
# /*******************************************************/

# /* check total copies of the book */
# -- your code

cursor.execute("""
SELECT COUNT(*) AS Total_Copies FROM Books
WHERE Title = 'Dracula';
""")
cursor.fetchall()

[(3,)]

In [5]:
# /* current total loans of the book */
# -- your code

cursor.execute("""
SELECT COUNT(*) AS Loaned_Copies FROM Loans AS l
INNER JOIN Books AS b ON l.BookID = b.BookID
WHERE b.Title = 'Dracula' AND l.ReturnedDate IS NULL;
""")
cursor.fetchall()

[(1,)]

In [6]:
# /* total available books of dracula */
# -- your code

cursor.execute("""
SELECT 
    (SELECT COUNT(*) AS Total_Copies FROM Books
	WHERE Title = 'Dracula') - 
    (SELECT COUNT(*) AS Loaned_Copies FROM Loans AS l
	INNER JOIN Books AS b ON l.BookID = b.BookID
	WHERE b.Title = 'Dracula' AND l.ReturnedDate IS NULL) AS Available_Copies;
""")
cursor.fetchall()

[(2,)]

In [7]:
# /*******************************************************/
# /* Add new books to the library                        */
# /*******************************************************/
# -- your code

cursor.execute("""
INSERT INTO Books VALUES (201, 'Rich Dad Poor Dad', 'Robert Kiyosaki', 1997, 1234567890);
""")
cursor.fetchall()

[]

In [8]:
# /*******************************************************/
# /* Check out Books: books(4043822646, 2855934983) whose patron_email(jvaan@wisdompets.com), loandate=2020-08-25, duedate=2020-09-08, loanid=by_your_choice*/
# /*******************************************************/
# -- your code

cursor.execute("""
INSERT INTO Loans (LoanID,BookID,PatronID,LoanDate,DueDate,ReturnedDate)
VALUES 
(
	2001,
	(SELECT BookID FROM Books WHERE Barcode = 4043822646),
	(SELECT PatronID FROM Patrons WHERE Email = 'jvaan@wisdompets.com'),
    '2020-08-25',
    '2020-09-08',
    NULL
),
(
	2002,
	(SELECT BookID FROM Books WHERE Barcode = 2855934983),
	(SELECT PatronID FROM Patrons WHERE Email = 'jvaan@wisdompets.com'),
    '2020-08-25',
    '2020-09-08',
    NULL
);
""")

cursor.fetchall()

[]

In [9]:
# /********************************************************/
# /* Check books for Due back                             */
# /* generate a report of books due back on July 13, 2020 */
# /* with patron contact information                      */
# /********************************************************/
# -- your code

cursor.execute("""
SELECT p.PatronID, p.FirstName, p.LastName, p.Email, b.BookID, b.Title, l.DueDate FROM Loans AS l
INNER JOIN Books AS b ON l.BookID = b.BookID
INNER JOIN Patrons AS p ON l.PatronID = p.PatronID
WHERE l.DueDate = '2020-07-13' AND l.ReturnedDate IS NULL;
""")

cursor.fetchall()

[(18,
  'Anthe',
  'Dinjes',
  'adinjesh@samoca.org',
  169,
  'Second Treatise of Government',
  '2020-07-13'),
 (47,
  'Lydie',
  'Awmack',
  'lawmack1a@samoca.org',
  155,
  'The Prophet',
  '2020-07-13'),
 (59,
  'Cornelia',
  'Koppe',
  'ckoppe1m@landonhotel.com',
  73,
  'Dracula',
  '2020-07-13'),
 (99,
  'Kala',
  'Simonian',
  'ksimonian2q@landonhotel.com',
  10,
  'The Count of Monte Cristo',
  '2020-07-13')]

In [10]:
# /*******************************************************/
# /* Return books to the library (which have barcode=6435968624) and return this book at this date(2020-07-05)                    */
# /*******************************************************/
# -- your code

cursor.execute("""
UPDATE Loans 
SET ReturnedDate = '2020-07-05'
WHERE BookID = (SELECT BookID FROM Books WHERE Barcode = 6435968624) AND ReturnedDate IS NULL;
""")

cursor.fetchall()

[]

In [11]:
# /*******************************************************/
# /* Encourage Patrons to check out books                */
# /* generate a report of showing 10 patrons who have
# checked out the fewest books.                          */
# /*******************************************************/
# -- your code

cursor.execute("""
SELECT p.PatronID, p.FirstName, p.LastName, p.Email, COUNT(l.LoanID) AS books_borrowed FROM Patrons AS p
LEFT JOIN Loans AS l ON p.PatronID = l.PatronID
GROUP BY p.PatronID
ORDER BY books_borrowed ASC
LIMIT 10;
""")

cursor.fetchall()

[(83, 'Caril', 'Matejic', 'cmatejic2a@orangevalleycaa.org', 9),
 (69, 'Alaster', 'Ruggles', 'aruggles1w@red30design.com', 9),
 (73, 'Merle', 'Sukbhans', 'msukbhans20@orangevalleycaa.org', 11),
 (64, 'Norby', 'Sleight', 'nsleight1r@kinetecoinc.com', 12),
 (94, 'Curr', 'Fittall', 'cfittall2l@orangevalleycaa.org', 12),
 (12, 'Orton', 'Stavers', 'ostaversb@orangevalleycaa.org', 14),
 (16, 'Carla', 'Swinfen', 'cswinfenf@kinetecoinc.com', 14),
 (7, 'Cammi', 'Kynett', 'ckynett6@orangevalleycaa.org', 14),
 (3, 'Aleen', 'Fasey', 'afasey2@kinetecoinc.com', 14),
 (21, 'Henri', 'Pughe', 'hpughek@orangevalleycaa.org', 15)]

In [12]:
# /*******************************************************/
# /* Find books to feature for an event                  
#  create a list of books from 1890s that are
#  currently available                                    */
# /*******************************************************/
# -- your code

cursor.execute("""
SELECT * FROM Books
WHERE Published BETWEEN 1890 AND 1899 AND BookID NOT IN (SELECT BookID FROM Loans WHERE ReturnedDate IS NULL);
""")

cursor.fetchall()

[(3,
  'The Adventures of Sherlock Holmes',
  'Arthur Conan Doyle',
  1892,
  2411676280),
 (7, 'The Sign of the Four', 'Arthur Conan Doyle', 1890, 9615072124),
 (8,
  'The Importance of Being Earnest: A Trivial Comedy for Serious People',
  'Oscar Wilde',
  1895,
  4733732465),
 (12, 'Dracula', 'Bram Stoker', 1897, 8232533734),
 (14,
  'The Awakening, and Selected Short Stories',
  'Kate Chopin',
  1899,
  3103139173),
 (15, 'The Time Machine', 'H. G. Wells', 1895, 9610930110),
 (17, 'Les Misérables', 'Victor Hugo', 1895, 8252831254),
 (18, 'The Jungle Book', 'Rudyard Kipling', 1894, 3655763446),
 (19, 'Heart of Darkness', 'Joseph Conrad', 1899, 9334362486),
 (26,
  'The Adventures of Sherlock Holmes',
  'Arthur Conan Doyle',
  1892,
  6117891368),
 (28, 'Heart of Darkness', 'Joseph Conrad', 1899, 2229312049),
 (43, 'The Yellow Wallpaper', 'Charlotte Perkins Gilman', 1892, 7476179474),
 (54, 'The Yellow Wallpaper', 'Charlotte Perkins Gilman', 1892, 2503974912),
 (55, 'An Occurrence at

In [13]:
# /*******************************************************/
# /* Book Statistics 
# /* create a report to show how many books were 
# published each year.                                    */
# /*******************************************************/

cursor.execute("""
SELECT Published, COUNT(DISTINCT(Title)) AS TotalNumberOfPublishedBooks FROM Books
GROUP BY Published
ORDER BY TotalNumberOfPublishedBooks DESC;
""")

cursor.fetchall()

[(1890, 3),
 (1895, 3),
 (1911, 2),
 (1899, 2),
 (1850, 2),
 (1898, 2),
 (1847, 2),
 (1859, 2),
 (1869, 2),
 (1892, 2),
 (1922, 2),
 (1884, 2),
 (1901, 1),
 (1894, 1),
 (1896, 1),
 (1897, 1),
 (1889, 1),
 (1887, 1),
 (1900, 1),
 (1880, 1),
 (1902, 1),
 (1905, 1),
 (1906, 1),
 (1908, 1),
 (1913, 1),
 (1914, 1),
 (1915, 1),
 (1920, 1),
 (1921, 1),
 (1923, 1),
 (1997, 1),
 (1839, 1),
 (1532, 1),
 (1605, 1),
 (1670, 1),
 (1689, 1),
 (1726, 1),
 (1729, 1),
 (1759, 1),
 (1791, 1),
 (1811, 1),
 (1813, 1),
 (1815, 1),
 (1818, 1),
 (1886, 1),
 (1849, 1),
 (1851, 1),
 (1852, 1),
 (1854, 1),
 (1858, 1),
 (1861, 1),
 (1865, 1),
 (1876, 1),
 (1878, 1),
 (1879, 1),
 (1375, 1),
 (1883, 1)]

In [14]:
# /*************************************************************/
# /* Book Statistics                                           */
# /* create a report to show 5 most popular Books to check out */
# /*************************************************************/

cursor.execute("""
SELECT b.BookID, b.Title, b.Author, b.Published, COUNT(l.LoanID) AS TotalTimesOfLoans FROM Books b
JOIN Loans l ON b.BookID = l.BookID
GROUP BY b.BookID
ORDER BY 4 DESC
LIMIT 5;
""")

cursor.fetchall()

[(78, 'The Prophet', 'Kahlil Gibran', 1923, 11),
 (155, 'The Prophet', 'Kahlil Gibran', 1923, 9),
 (77, 'The Prophet', 'Kahlil Gibran', 1923, 9),
 (69, 'Siddhartha', 'Hermann Hesse', 1922, 11),
 (184, 'Ulysses', 'James Joyce', 1922, 9)]

In [ ]:
# -- BONUS
# -- 15. Select shipper together with the total price of proceed orders

cursor.execute("""
SELECT s.ShipperName, SUM(od.Quantity * p.Price) AS Total_Price FROM orders as o
INNER JOIN shippers as s ON o.ShipperID = s.ShipperID
INNER JOIN order_details as od ON o.OrderID = od.OrderID
INNER JOIN products as p ON od.ProductID = p.ProductID
GROUP BY s.ShipperName;
""")

cursor.fetchall()